In [19]:
"""For basic init and call"""
import os

from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage
from dotenv import load_dotenv, find_dotenv

# .env 存储api_key
load_dotenv()
print(os.getenv("BAIDU_QIANFAN_CLINE_AK"))

NfbsdQ9hnvaXoz59Bc6L8bcE


In [20]:
llm = QianfanChatEndpoint(
    temperature=0.9,
    model="ernie-bot-turbo",
    qianfan_ak=os.getenv("BAIDU_QIANFAN_CLINE_AK"),
    qianfan_sk=os.getenv("BAIDU_QIANFAN_CLINE_SK"),
    verbose=True,
)

In [21]:
llm.invoke("你好！")

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'finish_reason': '', 'request_id': 'as-cqn1i96bh9', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 2, 'completion_tokens': 8, 'total_tokens': 10}, 'model_name': 'ernie-bot-turbo', 'finish_reason': 'stop', 'id': 'as-cqn1i96bh9', 'object': 'chat.completion', 'created': 1719757279, 'result': '你好！有什么我可以帮助你的吗？', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 2, 'completion_tokens': 8, 'total_tokens': 10}}, id='run-9e8016a2-9536-45f8-842d-72439dc98b5d-0')

In [22]:
#测试chatpromtTemplate
from langchain_core.prompts import ChatPromptTemplate

# 需要注意的一点是，这里需要指明具体的role，在这里是system和用户
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the technical writer"),
    ("user", "{input}")  # {input}为变量
])

In [23]:
# 我们可以把prompt和具体llm的调用和在一起（通过chain，chain可以理解为sequence of calls to take）
chain = prompt | llm 
chain.invoke({"input": "What is the Sora model?"})

AIMessage(content='Sora是一款由BOSCH（博世）公司开发的自行车变速器，它以其轻量化、高精度、高耐用性和易维护等特点而受到广泛关注。Sora变速器具有较小的体积和重量，能够提供良好的变速响应和稳定性，适合于各种骑行场景和骑行者需求。Sora变速器也被广泛应用于中档自行车的配置中，尤其是在长途骑行和通勤等场合中，它的可靠性和耐久性得到了广泛认可。\n\n同时，Sora是一个多义词，有时也可以指代一些与品牌相关的自行车品牌名称，以及用于表示与财务相关的情况等。具体使用时需要根据上下文语境进行理解和解释。', additional_kwargs={'finish_reason': '', 'request_id': 'as-8c8n8a2id8', 'object': 'chat.completion', 'search_info': [], 'function_call': {}, 'tool_calls': [{'type': 'function', 'function': {}}]}, response_metadata={'token_usage': {'prompt_tokens': 13, 'completion_tokens': 143, 'total_tokens': 156}, 'model_name': 'ernie-bot-turbo', 'finish_reason': 'stop', 'id': 'as-8c8n8a2id8', 'object': 'chat.completion', 'created': 1719757282, 'result': 'Sora是一款由BOSCH（博世）公司开发的自行车变速器，它以其轻量化、高精度、高耐用性和易维护等特点而受到广泛关注。Sora变速器具有较小的体积和重量，能够提供良好的变速响应和稳定性，适合于各种骑行场景和骑行者需求。Sora变速器也被广泛应用于中档自行车的配置中，尤其是在长途骑行和通勤等场合中，它的可靠性和耐久性得到了广泛认可。\n\n同时，Sora是一个多义词，有时也可以指代一些与品牌相关的自行车品牌名称，以及用于表示与财务相关的情况等。具体使用时需要根据上下文语境进行理解和解释。', 'is_truncated': False, 'need_clear_history

In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()  # 输出string
chain = prompt | llm | output_parser
chain.invoke({"input": "What is the Sora model?"})

'Sora模型是用于描述流体运动的基本数学模型，它是由Navier-Stokes方程组简化而来的。Sora模型主要考虑了流体运动的基本物理规律，如质量守恒、动量守恒和能量守恒等，可以用于描述流体运动的各种复杂情况，包括流动的稳定性、涡旋的运动和碰撞、射流等问题。该模型具有良好的普适性和可解释性，已经被广泛应用于航空航天、船舶制造、水利工程等许多领域。\n\n如果你有其他问题或需要进一步讨论，我将非常乐意回答！'

***RAG + Langchain***

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://baike.baidu.com/item/Sora/64060909")
docs = loader.load()
print(len(docs[0].page_content))


10018


In [26]:
#  使用openai embedding
# from langchain_wenxin import WenxinEmbeddings
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
embeddings = QianfanEmbeddingsEndpoint(qianfan_ak=os.getenv("BAIDU_QIANFAN_CLINE_AK"), qianfan_sk=os.getenv("BAIDU_QIANFAN_CLINE_SK"))

In [27]:
from langchain_community.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 使用 recursiveCharacterTextSplitter, 在春节前的课程中讲过其算法
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# 把docs切分成trunks，在这里只有一个doc，因为我们只抓取了一个页面；
documents = text_splitter.split_documents(docs)
print(documents[3].page_content)
# 存放在向量数据库中。把trunk转化成向量时候用的embedding工具为 wenxin embedding
vector = FAISS.from_documents(documents, embeddings)


[21]。但与此同时，民主倡导者和人工智能研究人员警告说，这些工具已经被用来欺骗和欺骗民众 [22]。名字由来播报编辑Sora在日语中是“天空”（そら）的意思，引申含义还有“自由”，象征着其无限的创造潜力 [50-51]。Sora 官网发展历程播报编辑模型发展2021年1月5日 文生图模型Dall-E发布 [27]Dall-E可以根据简单的描述创建逼真和清晰的图像，精通各种艺术风格，还可以生成文字制作建筑物上的标志，并制作同一场景的草图和全彩图像。2022年4月 Dall-E 2发布 [28]Dall-E 2不仅可以生成更真实和更准确的画像，而且能够将文本描述中的概念、属性和风格等元素综合起来，生成现实主义的图像和艺术作品。2022年11月30日 大语言模型ChatGPT发布 [29]ChatGPT不仅能与人对话，还能编写代码、创作内容等，这一款革命性产品的上线引发全球关注，上线仅5天用户数量就已突破100万。2023年3月15日 GPT-4正式面世 [30]GPT-4可以更准确地解决用户的难题，多模态的GPT-4还可以生成、编辑具有创意性或技术性的文章，在高级推理方面的表现超过其前代产品。展开全部正式发布美国当地时间2024年2月15日 [3]，OpenAI正式发布文生视频模型Sora [1]，并发布了48个文生视频案例和技术报告 [36]，正式入局视频生成领域


1. 给定input，从vector database搜索相似的documents（trunks）
2. documents加入到prompt里面（prompt template, 变量比如{context})
3. prompt call LLM， LLM返回response(答案)
4. 通过output parser得到格式化完之后的结果

In [28]:
# This chain takes a list of documents and formats them all into a prompt, then passes that prompt to an LLM.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [29]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
response = retrieval_chain.invoke({"input": "Sora是什么?"})
print(response["answer"])

Sora是一种人工智能文生视频大模型，由美国人工智能研究公司OpenAI发布。它可以根据用户的文本提示创建最长60秒的逼真视频，该模型了解这些物体在物理世界中的存在方式，可以深度模拟真实物理世界，能生成具有多个角色、包含特定运动的复杂场景。Sora的名称源于日文“空”（そら sora），即天空之意，以示其无限的创造潜力。


Agent***********

In [31]:
from langchain.tools.retriever import create_retriever_tool
#工具，工具名，功能描述
retriever_tool = create_retriever_tool(
    retriever,
    "Sora",
    "搜索有关Sora的信息. 对于有关sora的问题，都必须使用这个工具!",
)

In [32]:
tools = [retriever_tool]

In [33]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor


prompt = hub.pull("hwchase17/openai-functions-agent")
llm = QianfanChatEndpoint(model="ernie-bot-turbo", temperature=0.01,qianfan_ak=os.getenv("BAIDU_QIANFAN_CLINE_AK"), qianfan_sk=os.getenv("BAIDU_QIANFAN_CLINE_SK"), verbose=True)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
agent_executor.invoke({"input":"Sora是什么?"})

[WARNING][2024-06-30 22:21:33.703] base.py:829 [t:16308]: This key `functions` does not seem to be a parameter that the model `ernie-bot-turbo` will accept




> Entering new AgentExecutor chain...
Sora是一个英文单词，可以指Sora（角色），也可以是Sora（系列）。

作为名词时，Sora可以指代天空，也可以指代日本传说中的天空之境。同时，Sora也可以指代角色，例如在游戏《鬼泣》系列中，Sora是一个主角之一。

此外，Sora也可以指代一个系列，即Sora（系列）是日本游戏公司制作的一系列游戏。

因此，Sora的含义取决于上下文和语境。

> Finished chain.


{'input': 'Sora是什么?',
 'output': 'Sora是一个英文单词，可以指Sora（角色），也可以是Sora（系列）。\n\n作为名词时，Sora可以指代天空，也可以指代日本传说中的天空之境。同时，Sora也可以指代角色，例如在游戏《鬼泣》系列中，Sora是一个主角之一。\n\n此外，Sora也可以指代一个系列，即Sora（系列）是日本游戏公司制作的一系列游戏。\n\n因此，Sora的含义取决于上下文和语境。'}